 # Deeptech M&A Momentum: Trading Strategy Application
 
 ## Phase 4, Step 4.2: applying threshold logic
 
 This notebook takes the continuous, lagged sentiment signal (from Phase 3) and converts it into a discrete, actionable trading signal (+1 for Long, 0 for Neutral/Exit) based on pre-defined threshold rules.
 
 ---
 
 ### Setup and configuration

In [7]:
# Imports
from pathlib import Path
import sys

import polars as pl

In [ ]:
# --- Configuration ---
TEST_FREQUENCIES = ["1mo", "3mo", "6mo"]

# Trading Thresholds based on project brief logic
BUY_THRESHOLD = 0.70    # Go Long if signal is at or above this value
EXIT_THRESHOLD = 0.30   # Exit/Neutral if signal is at or below this value

# File paths
OUTPUT_DATA_DIR = Path("../../data/outputs")
OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)
print(f"Trading Signal Thresholds: Buy >= {BUY_THRESHOLD}, Exit/Neutral <= {EXIT_THRESHOLD}")

Trading Signal Thresholds: Buy >= 0.7, Exit/Neutral <= 0.3


### Step 4.2: Apply threshold logic and generate signal

In [ ]:
for freq in TEST_FREQUENCIES:
    print("\n" + "=" * 60)
    print(f"PHASE 4.2 | Generating Discrete Signal for Frequency: {freq}")
    print("=" * 60)
    
    # 1. Load the lagged sentiment data
    INPUT_PATH = OUTPUT_DATA_DIR / f"3.1_sentiment_signals_{freq}.csv"
    if not INPUT_PATH.exists():
        print(f"ERROR: Sentiment file not found for {freq} at {INPUT_PATH}. Skipping.")
        continue
        
    df_signal = pl.read_csv(INPUT_PATH)
    print(f"Loaded {len(df_signal):,} time-series rows.")
    
    # 2. Apply the Conditional Trading Rules using Polars' `when/then/otherwise`
    # We define the position (+1 for Long, 0 for Neutral).
    df_discrete_signal = df_signal.with_columns(
        pl.when(pl.col("lagged_sentiment_signal") >= BUY_THRESHOLD)
        .then(1) # Signal = +1 (Long Position)
        .when(pl.col("lagged_sentiment_signal") <= EXIT_THRESHOLD)
        .then(0) # Signal = 0 (Neutral/Exit Position)
        .otherwise(0) # Default to 0 (Neutral/Hold position if already long, or remain neutral)
        .cast(pl.Int8) # Cast to small integer type for efficiency
        .alias("discrete_signal")
    )
    
    print("  ✓ Discrete signal generated (+1, 0).")
    
    # 3. Filter out any periods where the signal is null (due to the 8-period rolling lag)
    df_final_signal = df_discrete_signal.filter(pl.col("discrete_signal").is_not_null())
    
    # Report signal distribution (how often we buy)
    buy_count = df_final_signal.filter(pl.col("discrete_signal") == 1).height
    total_periods = df_final_signal.height
    print(f"  Long Signal Frequency: {(buy_count / total_periods) * 100:.2f}%")
    
    # 4. Save the final signal series
    FINAL_OUTPUT_PATH = OUTPUT_DATA_DIR / f"4.0_discrete_signals_{freq}.csv"
    df_final_signal.write_csv(FINAL_OUTPUT_PATH)
    
    print(f"✓ Final discrete signal saved to: {FINAL_OUTPUT_PATH}")
    print(df_final_signal.head(5))

print("\n" + "=" * 60)
print("PHASE 4 COMPLETE: Discrete trading signals generated for all frequencies.")
print("Ready for Phase 5: Backtesting and Evaluation.")


PHASE 4.2 | Generating Discrete Signal for Frequency: 1mo
Loaded 728 time-series rows.
  ✓ Discrete signal generated (+1, 0).
  Long Signal Frequency: 13.60%
✓ Final discrete signal saved to: ..\..\data\outputs\4.1_trading_signals\4.1_discrete_signals_1mo.csv
shape: (5, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ announced_date ┆ deeptech_secto ┆ total_deal_vol ┆ transaction_c ┆ lagged_sentim ┆ discrete_sign │
│ ---            ┆ r              ┆ ume_usd        ┆ ount          ┆ ent_signal    ┆ al            │
│ str            ┆ ---            ┆ ---            ┆ ---           ┆ ---           ┆ ---           │
│                ┆ str            ┆ f64            ┆ i64           ┆ f64           ┆ i8            │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 2018-01-01     ┆ Advanced       ┆ 6.3282e8       ┆ 3             ┆ null          ┆ 0             │
│                ┆